In [ ]:
# =================================================================================================
# Authors: Andrei, Chathu, Mark
# Description: ------------
# =================================================================================================
# Version     |    Date      |      Author          |  Updates
# 0.1         |  23-Aug-2024 |    Andrei            |  Initial Version - Creating Git Repo
# 0.2         |  1-Aug-0000  |    Mark              |  
# 0.3         |  6-Aug-0000  |    Chathu            |  


In [1]:
print("Hello from assignmenttask2!")

Hello from assignmenttask2!
